# Scrapper para obtener información de búsqueda de google (LinkdIn y/o Facebook)

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import pandas as pd
from tqdm import tqdm

## Conexión al navegador

In [17]:
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)
start_url = 'https://google.com'
driver.get(start_url)
# Modificar el tamaño de la ventana
#driver.execute_script("document.body.style.zoom='50%'")

## Lectura de archivos csv para obtener nombres

In [29]:
df_filtered.shape

(28694,)

In [20]:
init=200
final=1000

df = pd.read_excel('Z:/Data/HSBC/Asignaciones/ASG_TDC_IA_ENERO_JUNIO.xlsx')
df['Nombre'] = df.Nombre.apply(lambda X: '"'+X.replace('/', ' ').replace('*', '').rstrip()+'"')
df_filtered = df['Nombre']
df_filtered.drop_duplicates(inplace=True)
df_input = pd.DataFrame(df_filtered).iloc[init:final]
df_input.head()

,Nombre
201,"""JUAN ANTONIO VEGA RODRIGUEZ"""
202,"""FRANCISCO JAVIER RODRIGUEZ RUIZ"""
203,"""CRISTIAN ADRIAN TOVAR TORRES"""
204,"""SANTIAGO EUSEBIO GONZALEZ"""
205,"""MA ISABEL BALDERAS RAMIREZ"""


In [ ]:
init=200
final=1000

df = pd.read_excel('Z:/Data/HSBC/Asignaciones/ASG_TDC_IA_ENERO_JUNIO.xlsx')
df['Nombre'] = df.Nombre.apply(lambda X: X.replace('/', ' ').replace('*', '').rstrip())
df_filtered = df['Nombre']
df_filtered.drop_duplicates(inplace=True)
df_input = pd.DataFrame(df_filtered).iloc[init:final]
df_input.head()

# Scrapper

## Concatenado de etiquetas

In [15]:

data_ = []
data = []
nombres = []

for i in tqdm(df_input['Nombre']):
    
    chrome_options = Options()
    driver = webdriver.Chrome(options=chrome_options)
    start_url = 'https://google.com'
    driver.get(start_url)
    
    nombres.append(i)
    wait = WebDriverWait(driver, 100000)
    nombre = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'gLFyf')))
    nombre.send_keys(Keys.CONTROL + "a")
    nombre.send_keys(Keys.DELETE)
    nombre.send_keys(i)
    nombre.send_keys(Keys.ENTER)

    # Esperar a que los resultados de búsqueda se carguen completamente
    time.sleep(2)

    # Obtener el HTML de la página actual
    html_link = driver.page_source
    soup = BeautifulSoup(html_link, 'html.parser')

    # Extracción de clases
    div = soup.find_all('div', class_="kvH3mc BToiNc UK95Uc")
    cont = soup.find_all('div', class_="Z26q7c UK95Uc")

    # Primer bucle para extraer la clase que conforma el bloque que corresponde al URL desplegado
    if div:
        for tag in div:
            href_tags = tag.find_all('a', href=True)
            text = tag.text

            for href_tag in href_tags:
                href = href_tag['href']

                data.append({'href': href, 'Text': text, 'Nombre': i})
    else:
        data.append({'href': None, 'Text': None, 'Nombre': i})

    # Segundo bucle para extraer los resúmenes de las URL desplegadas
    if cont:
        for tag in cont:
            text = ''

            span_tags = tag.find_all('span')
            if span_tags:
                for span_tag in span_tags:
                    if span_tag.get('class', []) == ['ITHCWe']:
                        continue
                    text += span_tag.text.strip() + ' '
            else:
                text = tag.text.strip()

            data_.append({'Text': text.strip()})
    else:
        data_.append({'Text': None})

    google = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="logo"]')))
    google.click()
    
    #driver.quit()

    # Rellenar los datos faltantes en data y data_ para tener la misma longitud
    max_length = max(len(data), len(data_))
    data += [{'href': None, 'Text': None}] * (max_length - len(data))
    data_ += [{'Text': None}] * (max_length - len(data_))

df = pd.DataFrame(data)
df_2 = pd.DataFrame(data_)

df_3 = pd.concat([df, df_2], axis=1)
df_3.dropna(inplace=True)
df_3 = df_3[df_3['href'].str.contains('facebook|twitter|instagram|linkedin|misprofesores|.org|tiktok', case=False)]
df_3 = df_3[~df_3['href'].str.contains('translate.google')]


# Se guarda el df en un csv
df_3.to_csv('Z:/Data/HSBC/SCRAPPING/200_1000_.csv')



  7%|▋         | 54/800 [08:12<1:53:23,  9.12s/it]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00EBA813+48355]
	(No symbol) [0x00E4C4B1]
	(No symbol) [0x00D55358]
	(No symbol) [0x00D3D293]
	(No symbol) [0x00D9E37B]
	(No symbol) [0x00DAC473]
	(No symbol) [0x00D9A536]
	(No symbol) [0x00D782DC]
	(No symbol) [0x00D793DD]
	GetHandleVerifier [0x0111AABD+2539405]
	GetHandleVerifier [0x0115A78F+2800735]
	GetHandleVerifier [0x0115456C+2775612]
	GetHandleVerifier [0x00F451E0+616112]
	(No symbol) [0x00E55F8C]
	(No symbol) [0x00E52328]
	(No symbol) [0x00E5240B]
	(No symbol) [0x00E44FF7]
	BaseThreadInitThunk [0x770805C9+25]
	RtlGetAppContainerNamedObjectPath [0x773E77AD+237]
	RtlGetAppContainerNamedObjectPath [0x773E777D+189]


In [ ]:
df_3

,href,Text,Nombre,Text
0,https://m.facebook.com/public/Oscar-Nu%C3%B1ez...,oscar-nuñez-maldonado Perfiles | Facebookfaceb...,OSCAR NUNEZ MALDONADO,Ver perfiles de personas llamadas Oscar Nuñez ...
1,https://www.facebook.com/oscar.n.maldonado/?lo...,Oscar Nuñez Maldonado | Facebookhttps://www.fa...,OSCAR NUNEZ MALDONADO,Oscar Nuñez Maldonado está en Facebook. Únete ...
2,https://mx.linkedin.com/in/oscar-maldonado-2b5...,Oscar Maldonado - instructor en seguridad publ...,OSCAR NUNEZ MALDONADO,Área metropolitana de Ciudad de México · instr...
3,https://mx.linkedin.com/in/oscar-maldonado-671...,Oscar Maldonado - FRAGANCIAS Y ESPECIALIDADESh...,OSCAR NUNEZ MALDONADO,"Guadalupe, Nuevo León, México · GERENTE GENERA..."
5,http://repositorio-digital.cide.edu/handle/116...,"Browsing Tesis by Author ""Núñez Maldonado, Jor...",OSCAR NUNEZ MALDONADO,"... Juárez Mendoza, Alma Cristina; Juárez Mond..."
...,...,...,...,...
1496,https://es-es.facebook.com/public/Jose-Manuel-...,jose-manuel-garcia-hernandez Perfiles - Facebo...,JOSE MANUEL GARCIA HERNANDEZ,Ver los perfiles de las personas que se llaman...
1497,https://www.facebook.com/people/Jos%C3%A9-Manu...,José Manuel García Hernández - Facebookhttps:/...,JOSE MANUEL GARCIA HERNANDEZ,José Manuel García Hernández. 5 Me gusta • 5 s...
1501,https://mx.linkedin.com/in/mar%C3%ADa-constant...,María Constantina Lopez Cortez - Exfarma S.A. ...,CONSTANTINA LOPEZ RODRIGUEZ,"Xalapa, Veracruz de Ignacio de la Llave, Méxic..."
1502,https://www.instagram.com/guille_lr9/?hl=es-la,Guillermo López Rodríguez (@guille_lr9) • Inst...,CONSTANTINA LOPEZ RODRIGUEZ,"Constantina, Sevilla Lo único imposible es aqu..."


In [ ]:
captcha = driver.find_element(By.CLASS_NAME, 'gLFyf')
captcha.click()